In [1]:
from pathlib import Path

data_path = Path.cwd() / "lionagi_data"  # Path to the data directory

In [2]:
import lionagi as li

### Prepare QA Tool

In [3]:
docs = li.load(
    input_dir=data_path, recursive=True, required_exts=[".py"], to_lion=False
)

docs = [i for i in docs if len(i.text) > 100]

# chunks = li.chunk(
#     docs, chunker = "CodeSplitter", chunker_type = "llama_index",
#     to_lion=False,
#     chunker_kwargs = {
#         "language": "python",
#         "chunk_lines": 100,
#         "chunk_lines_overlap": 10,
#         "max_chars": 2000,},
# )

In [4]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(
    model=OpenAIEmbeddingModelType.TEXT_EMBED_3_LARGE
)

In [5]:
# from llama_index.core import VectorStoreIndex

# index = VectorStoreIndex(chunks)
# index.storage_context.persist(persist_dir="./lionagi_index")

In [6]:
from llama_index.core import load_index_from_storage, StorageContext

index_id = "91fe61e0-89b5-4202-acff-435707e60119"

storage_context = StorageContext.from_defaults(persist_dir="./lionagi_index")
index = load_index_from_storage(storage_context, index_id=index_id)

In [7]:
from llama_index.core.postprocessor import LLMRerank

reranker = LLMRerank(choice_batch_size=10, top_n=5)
query_engine = index.as_query_engine(node_postprocessors=[reranker])

In [8]:
source_codes_responses = []


async def query_codebase(query):
    """
    Perform a query to a QA bot with access to a vector index built with package lionagi codebase

    Args:
        query (str): The query string to search for in the LionAGI codebase.

    Returns:
        str: The string representation of the response content from the codebase query.
    """
    response = await query_engine.aquery(query)
    source_codes_responses.append(response)
    return str(response.response)

### Construct Workflow

In [9]:
instruction = """
write a good API documentation for this code, must use query 
engine to check meanings of related code concepts to accurately describe, 
for example if a name is used but not present in context, you must use the query engine.
You must integrate the information from query engine to verify the correctness 
of the documentation. make sure to cross reference the code with the
query engine to ensure the documentation is accurate
"""

edit = """
provide documentation only: final documentation in md format, do not include other fields
do not present in JSON format, only markdown format
you asked a lot of good questions and got plenty answers, please integrate your 
conversation, be a lot more technical, you will be rewarded with 500 dollars for 
great work, and punished for subpar work, take a deep breath, you can do it
"""

In [10]:
from PROMPTS import sys_prompt  # put your system prompt here

tools = li.func_to_tool(query_codebase)

model = li.iModel(
    model="gpt-4o",
    provider="openai",
    interval_tokens=1_000_000,
    interval_requests=1_000,
    interval=60,
)


async def write_doc(context):
    branch = li.Branch(system=sys_prompt, tools=[query_codebase], imodel=model)

    form = await branch.direct(
        instruction=instruction,
        context=context,
        reason=True,
        score=True,
        allow_action=True,
        allow_extension=True,
        max_extension=2,
    )
    
    if form is None:
        return None, None

    form.answer = await branch.chat(
        instruction=edit,
        temperature=0.5,
    )
    
    # save all messages into a unique file
    df = branch.to_df()
    df.to_csv(f"lion_doc_{branch.ln_id[:8]}.csv", index=False)

    return form, branch


contexts = [i.text for i in docs]

### Run workflow

In [11]:
form, branch = await write_doc(contexts[74])

Attempt 1/3 failed: 'UnitForm' object has no attribute 'instruction', retrying...


In [12]:
form.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: 
write a good API documentation for this code, must use query 
engine to check meanings of related code concepts to accurately describe, 
for example if a name is used but not present in context, you must use the query engine.
You must integrate the information from query engine to verify the correctness 
of the documentation. make sure to cross reference the code with the
query engine to ensure the documentation is accurate

- Additional context: """
Copyright 2024 HaiyangLi

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

from lionagi.core.collections.abc import Component
from lionagi.core.collections import (
    Pile,
    Progression,
    Flow,
    progression,
    pile,
    flow,
    iModel,
)
from lionagi.core.message import System
from lionagi.libs.ln_api import BaseService
from typing import Any, Tuple
from lionagi.core.action.tool import Tool, TOOL_TYPE
from lionagi.core.action.tool_manager import ToolManager

from lionagi.libs import SysUtil


class Session:

    def __init__(
        self,
        system=None,  # the default system message node for the session
        model=None,
        datalogger=None,
        persist_path=None,
    ):

        self.ln_id = SysUtil.create_id()
        self.timestamp = SysUtil.get_timestamp(sep=None)[:-6]
        self.system = system or System(
            system="You are a helpful assistant.",
            sender=self.ln_id,
            recipient="assistant",
        )
   ...

**reason**: 
 Let's think step by step. We have gathered detailed information about the classes and methods used in the code. Now, we can create a comprehensive API documentation that accurately describes the functionality and purpose of each component.

**actions**: 

**action_required**: False

**extension_required**: False

**score**: 9

**available_tools**: ['query_codebase']

**answer**: 
 ### API Documentation for `Session` Class

#### Class: `Session`

**Description**:
The `Session` class manages different branches of operations, facilitating the collection and distribution of data or requests across these branches. It initializes with various configurations and provides methods for branch manipulation, data collection, and data sending.

#### Method: `__init__`

**Signature**:
```python
def __init__(
    self,
    system: dict | list | System | None = None,
    sender: str | None = None,
    llmconfig: dict[str, str | int | dict] | None = None,
    service: BaseService | None = None,
    branches: dict[str, Branch] | None = None,
    default_branch: Branch | None = None,
    default_branch_name: str | None = None,
    tools: TOOL_TYPE | None = None,
    tool_manager: ToolManager | None = None,
    messages: dataframe.ln_DataFrame | None = None,
    datalogger: None | DataLogger = None,
    persist_path: Path | str | None = None,
)
```

**Parameters**:
- `system` (dict | list | System | None): The default system message node for the session.
- `sender` (str | None): The default sender identifier.
- `llmconfig` (dict[str, str | int | dict] | None): Configuration for language learning models.
- `service` (BaseService | None): External service instance.
- `branches` (dict[str, Branch] | None): Dictionary of branches.
- `default_branch` (Branch | None): The default branch instance.
- `default_branch_name` (str | None): The name of the default branch.
- `tools` (TOOL_TYPE | None): List of tools available for the session.
- `tool_manager` (ToolManager | None): Tool manager for handling tools.
- `messages` (dataframe.ln_DataFrame | None): Initial set of messages.
- `datalogger` (None | DataLogger): Data logger instance.
- `persist_path` (Path | str | None): Path for persistence.

**Return Values**:
- None

#### Method: `new_branch`

**Signature**:
```python
def new_branch(
    self,
    branch_name: str,
    system: dict | list | System | None = None,
    sender: str | None = None,
    messages: dataframe.ln_DataFrame | None = None,
    tool_manager=None,
    service=None,
    llmconfig=None,
    tools: TOOL_TYPE = False,
) -> None:
```

**Parameters**:
- `branch_name` (str): Name of the new branch.
- `system` (dict | list | System | None): System or context identifier for the new branch.
- `sender` (str | None): Default sender identifier for the new branch.
- `messages` (dataframe.ln_DataFrame | None): Initial set of messages for the new branch.
- `tool_manager` (Optional): Tool manager for handling tools in the new branch.
- `service` (BaseService | None): External service instance for the new branch.
- `llmconfig` (dict[str, Any] | None): Configuration for language learning models in the new branch.
- `tools` (TOOL_TYPE | None): List of tools available for the new branch.

**Return Values**:
- None

**Exceptions Raised**:
- `ValueError`: If the branch name already exists.

**Usage Examples**:
```python
# Example 1: Create a new branch with default configurations
session.new_branch("new_branch_name")

# Example 2: Create a new branch with specific configurations
session.new_branch(
    branch_name="new_branch_name",
    system={"system": "New system"},
    sender="new_sender",
    messages=None,
    tool_manager=None,
    service=None,
    llmconfig=None,
    tools=None
)
```

#### Method: `get_branch`

**Signature**:
```python
def get_branch(
    self, branch: Branch | str | None = None, get_name: bool = False
) -> Branch | Tuple[Branch, str]:
```

**Parameters**:
- `branch` (Branch | str | None): The branch name or instance to retrieve.
- `get_name` (bool): If True, returns a tuple of the branch instance and its name.

**Return Values**:
- `Branch | Tuple[Branch, str]`: The branch instance or a tuple of the branch instance and its name.

**Exceptions Raised**:
- `ValueError`: If the branch name does not exist or the branch input is invalid.

**Usage Examples**:
```python
# Example 1: Retrieve a branch by name
branch_instance = session.get_branch("existing_branch_name")

# Example 2: Retrieve a branch by name and get its name
branch_instance, branch_name = session.get_branch("existing_branch_name", get_name=True)
```

#### Method: `change_default_branch`

**Signature**:
```python
def change_default_branch(self, branch: str | Branch) -> None:
```

**Parameters**:
- `branch` (str | Branch): The branch name or instance to set as the new default.

**Return Values**:
- None

**Usage Examples**:
```python
# Example: Change the default branch
session.change_default_branch("new_default_branch")
```

#### Method: `delete_branch`

**Signature**:
```python
def delete_branch(self, branch: Branch | str, verbose: bool = True) -> bool:
```

**Parameters**:
- `branch` (Branch | str): The branch name or instance to delete.
- `verbose` (bool): If True, prints a message upon deletion.

**Return Values**:
- `bool`: True if the branch was successfully deleted.

**Exceptions Raised**:
- `ValueError`: If attempting to delete the current default branch.

**Usage Examples**:
```python
# Example: Delete a branch
session.delete_branch("branch_to_delete")
```

#### Method: `merge_branch`

**Signature**:
```python
def merge_branch(
    self,
    from_: str | Branch,
    to_branch: str | Branch,
    update: bool = True,
    del_: bool = False,
) -> None:
```

**Parameters**:
- `from_` (str | Branch): The source branch name or instance.
- `to_branch` (str | Branch): The target branch name or instance where the merge will happen.
- `update` (bool): If True, updates the target branch with the source branch's settings.
- `del_` (bool): If True, deletes the source branch after merging.

**Return Values**:
- None

**Usage Examples**:
```python
# Example: Merge one branch into another and delete the source branch
session.merge_branch("source_branch", "target_branch", del_=True)
```

#### Method: `collect`

**Signature**:
```python
def collect(self, from_: str | Branch | list[str | Branch] | None = None):
```

**Parameters**:
- `from_` (str | Branch | list[str | Branch] | None): The branch(es) from which to collect requests. Can be a single branch name, a single branch instance, a list of branch names, a list of branch instances, or None. If None, requests are collected from all branches.

**Return Values**:
- None

**Usage Examples**:
```python
# Example 1: Collect requests from a specific branch
session.collect("branch_name")

# Example 2: Collect requests from multiple branches
session.collect([branch_instance_1, "branch_name_2"])

# Example 3: Collect requests from all branches
session.collect()
```

#### Method: `send`

**Signature**:
```python
def send(self, to_: str | Branch | list[str | Branch] | None = None):
```

**Parameters**:
- `to_` (str | Branch | list[str | Branch] | None): The target branch(es) to which to send requests. Can be a single branch name, a single branch instance, a list of branch names, a list of branch instances, or None. If None, requests are sent to all branches.

**Return Values**:
- None

**Usage Examples**:
```python
# Example 1: Send requests to a specific branch
session.send("target_branch")

# Example 2: Send requests to multiple branches
session.send([branch_instance_1, "target_branch_2"])

# Example 3: Send requests to all branches
session.send()
```

#### Method: `collect_send_all`

**Signature**:
```python
def collect_send_all(self, receive_all=False):
```

**Parameters**:
- `receive_all` (bool): If True, triggers a `receive_all` method on each branch after sending requests, which can be used to process or acknowledge the received data.

**Return Values**:
- None

**Usage Examples**:
```python
# Example 1: Collect and send requests across all branches
session.collect_send_all()

# Example 2: Collect and send requests across all branches and invoke receive operation
session.collect_send_all(receive_all=True)
```

#### Method: `setup_default_branch`

**Signature**:
```python
def setup_default_branch(self, **kwargs):
```

**Parameters**:
- `**kwargs`: Keyword arguments for setting up the default branch.

**Return Values**:
- None

#### Method: `_verify_default_branch`

**Signature**:
```python
def _verify_default_branch(self):
```

**Parameters**:
- None

**Return Values**:
- None

#### Method: `_setup_default_branch`

**Signature**:
```python
def _setup_default_branch(
    self,
    system,
    sender,
    default_branch,
    default_branch_name,
    messages,
    tool_manager,
    service,
    llmconfig,
    tools,
    persist_path,
    datalogger,
):
```

**Parameters**:
- `system`: System configuration.
- `sender`: Sender identifier.
- `default_branch`: Default branch instance.
- `default_branch_name`: Name of the default branch.
- `messages`: Initial set of messages.
- `tool_manager`: Tool manager for handling tools.
- `service`: External service instance.
- `llmconfig`: Configuration for language learning models.
- `tools`: List of tools available for the branch.
- `persist_path`: Path for persistence.
- `datalogger`: Data logger instance.

**Return Values**:
- None

In [13]:
branch.to_df()

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,629bc2b34fa9bc8de2a1b21e2cb98efb,System,2024-05-22T20:58:56.975942,system,"{'system_info': ' you are a helpful assistant,...",{'last_updated': {'recipient': '2024-05-22T20:...,system,3fb8ce811d619b2b824bd756514f5b1a
1,ed8c70b0f79c2fa9f056f6bf4f7aed73,Instruction,2024-05-22T20:58:56.976611,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-22T20:58:...,user,3fb8ce811d619b2b824bd756514f5b1a
2,995fdd82b9fb919e99366480192da907,AssistantResponse,2024-05-22T20:59:06.102102,assistant,"{'assistant_response': '```json { ""answer"": ...",{'last_updated': {'sender': '2024-05-22T20:59:...,3fb8ce811d619b2b824bd756514f5b1a,user
3,7ed576ef71c5a4817e2e38d53ef49b71,ActionRequest,2024-05-22T20:59:06.105624,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T20:5...,3fb8ce811d619b2b824bd756514f5b1a,0aee0da97a38d6cea9606fe58d1b736d
4,b49e61d4b55b9f6bb7cfadf1b8d1c0ee,ActionRequest,2024-05-22T20:59:06.105875,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T20:5...,3fb8ce811d619b2b824bd756514f5b1a,0aee0da97a38d6cea9606fe58d1b736d
5,40bd6846d275f118caa5798314b486a7,ActionRequest,2024-05-22T20:59:06.106026,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T20:5...,3fb8ce811d619b2b824bd756514f5b1a,0aee0da97a38d6cea9606fe58d1b736d
6,da7ea25fc598de05f676bf6c72e581e5,ActionRequest,2024-05-22T20:59:06.106163,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T20:5...,3fb8ce811d619b2b824bd756514f5b1a,0aee0da97a38d6cea9606fe58d1b736d
7,b6dcfb22cfd007e21e69e2af62470f52,ActionRequest,2024-05-22T20:59:06.106296,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T20:5...,3fb8ce811d619b2b824bd756514f5b1a,0aee0da97a38d6cea9606fe58d1b736d
8,4e00bc4e775ab847e33e84c4f3bf22c8,ActionRequest,2024-05-22T20:59:06.106429,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T20:5...,3fb8ce811d619b2b824bd756514f5b1a,0aee0da97a38d6cea9606fe58d1b736d
9,ba76f1fc330d64671ec4855c115b72dc,ActionRequest,2024-05-22T20:59:06.106560,assistant,{'action_request': {'function': 'query_codebas...,{'last_updated': {'function': '2024-05-22T20:5...,3fb8ce811d619b2b824bd756514f5b1a,0aee0da97a38d6cea9606fe58d1b736d


In [14]:
msg = branch.messages[2]

In [15]:
form.tool_schema

{'tools': [{'type': 'function',
   'function': {'name': 'query_codebase',
    'description': 'Perform a query to a QA bot with access to a vector index built with package lionagi codebase',
    'parameters': {'type': 'object',
     'properties': {'query': {'type': 'string',
       'description': 'The query string to search for in the LionAGI codebase.'}},
     'required': ['query']}}}]}

In [16]:
form

ln_id                                  efa9a4d8505fa41ef68578a10bd5ff1c
created                                      2024-05-22T20:59:16.679879
metadata              {'last_updated': {'input_fields': '2024-05-22T...
content                                                            None
template_name                                             UnitDirective
assignment                                               task -> answer
input_fields                                        [task, tool_schema]
requested_fields      [answer, reason, extension_required, actions, ...
task                  Follow the prompt and provide the necessary ou...
validation_kwargs     {'reason': {}, 'extension_required': {}, 'acti...
confidence_score                                                   None
reason                Let's think step by step. We have gathered det...
actions                                                              {}
action_required                                                 